In [ ]:
import tensorflow as tf
import numpy as np
import os
import time

In [1]:
pwd

'C:\\Users\\Ravi kumar'

In [1]:
path_to_file = r"C:\Users\Ravi kumar\Downloads\riscv_arithmetic_basic_test_4.S"

In [2]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 486992 characters


In [3]:
# Take a look at the first 250 characters in text
print(text[:250])

.include "user_define.h"
.globl _start
.section .text
_start:           
                  .include "user_init.s"
                  csrr x5, 0xf14
                  li x6, 0
                  beq x5, x6, 0f

0: la x29, h0_start
jalr x0, x29, 0
h0_sta


In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

67 unique characters


In [7]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Process the text
Vectorize the text
Before training, you need to convert the strings to a numerical representation.

The tf.keras.layers.StringLookup layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [8]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [9]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[42, 43, 44, 45, 46, 47, 48], [65, 66, 67]]>

In [10]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [11]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [12]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [13]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [14]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(486992,), dtype=int64, numpy=array([ 9, 50, 55, ..., 65, 10,  1], dtype=int64)>

In [15]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [16]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

.
i
n
c
l
u
d
e
 
"


In [17]:
seq_length = 400
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
    print(chars_from_ids(seq))

tf.Tensor(
[b'.' b'i' b'n' b'c' b'l' b'u' b'd' b'e' b' ' b'"' b'u' b's' b'e' b'r'
 b'_' b'd' b'e' b'f' b'i' b'n' b'e' b'.' b'h' b'"' b'\n' b'.' b'g' b'l'
 b'o' b'b' b'l' b' ' b'_' b's' b't' b'a' b'r' b't' b'\n' b'.' b's' b'e'
 b'c' b't' b'i' b'o' b'n' b' ' b'.' b't' b'e' b'x' b't' b'\n' b'_' b's'
 b't' b'a' b'r' b't' b':' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' '
 b' ' b' ' b'\n' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' '
 b' ' b' ' b' ' b' ' b' ' b' ' b' ' b'.' b'i' b'n' b'c' b'l' b'u' b'd'
 b'e' b' ' b'"' b'u' b's' b'e' b'r' b'_' b'i' b'n' b'i' b't' b'.' b's'
 b'"' b'\n' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' '
 b' ' b' ' b' ' b' ' b' ' b' ' b'c' b's' b'r' b'r' b' ' b'x' b'5' b','
 b' ' b'0' b'x' b'f' b'1' b'4' b'\n' b' ' b' ' b' ' b' ' b' ' b' ' b' '
 b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b'l' b'i' b' '
 b'x' b'6' b',' b' ' b'0' b'\n' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' '
 b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b'b' b'e

In [18]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'.include "user_define.h"\n.globl _start\n.section .text\n_start:           \n                  .include "user_init.s"\n                  csrr x5, 0xf14\n                  li x6, 0\n                  beq x5, x6, 0f\n\n0: la x29, h0_start\njalr x0, x29, 0\nh0_start:\n                  li x18, 0x40001104\n                  csrw 0x301, x18\nkernel_sp:        \n                  la x13, kernel_stack_end\n\ntrap_vec_init'
b':    \n                  la x18, mtvec_handler\n                  ori x18, x18, 0\n                  csrw 0x305, x18 # MTVEC\n\nmepc_setup:       \n                  la x18, init\n                  csrw 0x341, x18\n\ncustom_csr_setup: \n                  nop\n\ninit_machine_mode:\n                  li x18, 0x1800\n                  csrw 0x300, x18 # MSTATUS\n                  li x18, 0x0\n                  csrw 0x'
b'304, x18 # MIE\n                  mret\ninit:             \n                  li x0, 0xf1b0ea1f\n                  li x1, 0xfea23c14\n                  li 

In [19]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [20]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [21]:
dataset = sequences.map(split_input_target)

In [22]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'.include "user_define.h"\n.globl _start\n.section .text\n_start:           \n                  .include "user_init.s"\n                  csrr x5, 0xf14\n                  li x6, 0\n                  beq x5, x6, 0f\n\n0: la x29, h0_start\njalr x0, x29, 0\nh0_start:\n                  li x18, 0x40001104\n                  csrw 0x301, x18\nkernel_sp:        \n                  la x13, kernel_stack_end\n\ntrap_vec_ini'
Target: b'include "user_define.h"\n.globl _start\n.section .text\n_start:           \n                  .include "user_init.s"\n                  csrr x5, 0xf14\n                  li x6, 0\n                  beq x5, x6, 0f\n\n0: la x29, h0_start\njalr x0, x29, 0\nh0_start:\n                  li x18, 0x40001104\n                  csrw 0x301, x18\nkernel_sp:        \n                  la x13, kernel_stack_end\n\ntrap_vec_init'


In [23]:
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 400), dtype=tf.int64, name=None), TensorSpec(shape=(64, 400), dtype=tf.int64, name=None))>

In [24]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [25]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [26]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [27]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 400, 68) # (batch_size, sequence_length, vocab_size)


In [28]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  17408     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  69700     
                                                                 
Total params: 4,025,412
Trainable params: 4,025,412
Non-trainable params: 0
_________________________________________________________________


In [29]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [30]:
sampled_indices

array([ 5, 45, 64, 55, 10, 40, 52, 18, 46, 41, 57, 31, 24, 45, 10, 33, 51,
       33, 13, 35, 65, 25, 58, 20, 35, 20, 29, 27, 40, 26,  7, 51,  3, 29,
       52, 65, 56, 39, 55, 10,  5, 44, 53, 28, 65, 57, 39, 53, 61, 34, 26,
        0, 20, 22, 32, 18, 15, 52, 52, 35, 62, 47, 41, 56, 29, 59, 23,  6,
       32, 51, 35, 57, 23, 11, 10, 26,  8, 32, 31, 52, 50,  6, 50, 14, 59,
       41, 33,  4,  3, 28,  2, 39, 28,  4, 41, 24, 32, 25,  3, 66, 16, 47,
       53, 12, 26,  0, 55, 20, 19, 62, 38, 16, 18, 62, 37, 22,  5, 21, 24,
       18,  8, 60, 51, 55, 62, 28, 55, 19, 26, 51, 21, 46, 15, 18, 15, 23,
       37, 43, 49, 46, 19, 39, 19,  9,  6, 31, 37,  5, 58, 34, 15, 35,  0,
       32, 31, 50, 64,  7,  4, 59, 11, 53,  9, 11, 15,  7, 49, 38, 66, 63,
       11, 54, 67, 29, 54, 45, 58,  0, 12, 19, 51, 64, 37, 37,  3, 52,  2,
       14, 67, 25,  6, 23, 44, 11, 37, 58, 45, 41, 42, 41, 60, 52, 44, 21,
        2, 37, 53, 49, 67,  4, 57, 48, 35, 13, 15, 44,  1, 60, 58, 38, 30,
       47, 22, 19, 25, 56

In [31]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b', gp\n                  mulh         a2, t6, a6\n                  c.nop\n                  divu         gp, ra, s5\n                  sll          t1, t0, t5\n                  c.xor        a1, a2\n                  div          a1, gp, s7\n                  mulhsu       t2, a2, s0\n                  mulhsu       t1, a1, s7\n                  c.slli       t3, 30\n                  c.srli       a1, 25\n     '

Next Char Predictions:
 b'(dwn0Vk8e_pLBd0NjN3PxCq:P:IEVD,j"IkxoUn0(clGxpUltOD[UNK]:@M85kkPuf_oIrA)MjPpA10D-MLki)i4r_N#"G UG#_BMC"y6fl2D[UNK]n:9uT68uS@(;B8-sjnuGn9Dj;e585ASbhe9U9.)LS(qO5P[UNK]MLiw,#r1l.15,hTyv1mzImdq[UNK]29jwSS"k 4zC)Ac1Sqd_a_skc; Slhz#pgP35c\nsqTKf@9Cou.Gjpl9GbtArLxrs3_Oz;b"KBl 20g 4U3RMUVS-@eTo;BTcC;SKL2@sNyMGSxwBG,N7wB#aaSVRVksVSOEyc"A(9x.faDCLKdqjaE12K1#ngaGwR.qPiRr\nmaNtgAEAe,h[UNK]n0N#\ny\n_5sII_7Sa65yGn"Ky. N3#VeNqNlAf-7P@E'


In [32]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [33]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 400, 68)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.225536, shape=(), dtype=float32)


In [34]:
tf.exp(example_batch_mean_loss).numpy()

68.411156

In [35]:
model.compile(optimizer='adam', loss=loss)

In [36]:
import os

In [37]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [38]:
EPOCHS=20
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
18/18 [==============================] - 198s 11s/step - loss: 3.1897
Epoch 2/20
18/18 [==============================] - 212s 12s/step - loss: 1.4539
Epoch 3/20
18/18 [==============================] - 221s 12s/step - loss: 1.2489
Epoch 4/20
18/18 [==============================] - 214s 12s/step - loss: 1.0908
Epoch 5/20
18/18 [==============================] - 196s 11s/step - loss: 0.9354
Epoch 6/20
18/18 [==============================] - 198s 11s/step - loss: 0.8133
Epoch 7/20
18/18 [==============================] - 214s 12s/step - loss: 0.7195
Epoch 8/20
18/18 [==============================] - 213s 12s/step - loss: 0.6522
Epoch 9/20
18/18 [==============================] - 215s 12s/step - loss: 0.5973
Epoch 10/20
18/18 [==============================] - 203s 11s/step - loss: 0.5500
Epoch 11/20
18/18 [==============================] - 211s 12s/step - loss: 0.5095
Epoch 12/20
18/18 [==============================] - 206s 11s/step - loss: 0.4888
Epoch 13/20
18/18 [======

In [39]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [40]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [41]:
start = time.time()
states = None
next_char = tf.constant(['user_define.h'])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

user_define.h_st_nta, s6f
                 rem          s5, 579018
                  lw  x17, 00(x25
                  lw   x16, 00(x20)
                  c.lw 0x3710, 31s
                  li x19, x1O, 0
                  li           55, 0x3
                  srai x18, x10, x80
                  rem x22, 0
                  lemu          t6, 0x3605000
                  li           st, 0x00b                  re, x28, 800
                  lw  x10, 60(x20)
                  lw  x26, 700(x28)
                  sw x10, 114(x20)
                  rem x29, 2
                   srli x26, 0x80026a3, a1
                  jslliux x2, a1, 29
                  slli         s11, t1, zero
                  slt          t0, ra, s20
                   mulh         t0, s7, a4
                  addi         s0, a4, 998
                  c.li         s0, 23
                  addi         s8, a1, 4779
                  c.add        t3, s1
                  c.sub        s5, a1
                  c.xor   

In [42]:
start = time.time()
states = None
next_char = tf.constant(['kernel_sp:'])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

kernel_sp: 22
                 div          s1, s9, t1
                   c.li         s9, 21
                  c.addi16sp   sp, -16
                  slt          s7, s0, a5
                  srli         s9, s2, s9
                  c.srli       a0, 8
                  add          a0, a6, t4
                  c.or         tp, a3
                  remu         t2, s6, tp
                  slli         s0, s2, -894
                  mulh          a4, s10, s1
                  srli         t1, gp, -1180
                  c.srai       s1, 2
                  c.sp, sp, -3
                  c.sub        s7, a0
                  c.slli       a5, 13
                  mulh         t1, a1, s4
                  nop
                  c.li         s1, -1
                  or           t6, gp, s4
                   c.slli       a0, 32
                  c.or         s8, a5
                  divu         s10, gp, s6
                  c.srli       a1, 18
                  c.lui        a1, 37
       